In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

#Sept 9

In [ ]:
powerDF = pd.read_csv('2022-09-09-Power.csv')
del powerDF['Unnamed: 0']
powerDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,8.816639,5.509179,6.402390,7.678388
1,6.377071,9.253333,7.134431,10.658632
2,5.261156,8.733055,5.330680,9.883824
3,5.392678,9.824773,8.610629,12.035916
4,8.381074,10.230964,8.598232,10.655943


In [ ]:
praatAcousticFeatsDf = pd.read_csv('noModeratorSpeech_Sept9PerTwoSecondPraatFeatures.csv')
del praatAcousticFeatsDf['Unnamed: 0'], praatAcousticFeatsDf['Time (secs)']
praatAcousticFeatsDf.head()

,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
powerDF.shape, praatAcousticFeatsDf.shape

((1155, 4), (568, 9))

In [ ]:
# Reshape the array to have 50 columns (assuming the number of rows is a multiple of 50)
dfArray = powerDF[:-1].to_numpy()
reshaped_array = dfArray.reshape(-1, 2, dfArray.shape[1])

# Calculate the mean along axis 1 (average every 5 rows)
averaged_array = reshaped_array.mean(axis=1)

# Create a new DataFrame from the averaged array
columns = powerDF.columns  # Assuming you want to keep the same columns
resampled_powerDF = pd.DataFrame(averaged_array, columns=columns)
resampled_powerDF.shape

(577, 4)

In [ ]:
resampled_powerDF = resampled_powerDF.iloc[:praatAcousticFeatsDf.shape[0]]
resampled_powerDF.shape

(568, 4)

In [ ]:
powerPraatDF = pd.concat([resampled_powerDF, praatAcousticFeatsDf], axis=1)
powerPraatDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,7.596855,7.381256,6.768410,9.168510,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,5.326917,9.278914,6.970654,10.959870,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,7.308063,7.351847,7.363880,7.966896,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,8.009013,10.256529,8.385252,6.449097,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,8.119956,6.437368,8.964941,9.929328,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.1081
rValFor95ConfidenceLevel = 0.0823

##random noise t-test (1)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random 1136-element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerPraatDF.shape[0])


568

In [ ]:
powerPraatDF['Noise'] = random_array

In [ ]:
powerPraatDF.corr()['Noise'][4:-1]#.abs()

speechrate(nsyll / dur)                               -0.015869
Average Syllable Duration (speakingtot/voicedcount)    0.010459
articulation rate(nsyll / phonationtime)              -0.002522
Speech-to-pause ratio                                  0.016999
Mean Syllable Period                                   0.016700
Variability of Syllable Period                        -0.083157
Mean Pause Duration                                   -0.001641
Variability of Pause Duration                          0.034017
# of pauses                                            0.050493
Name: Noise, dtype: float64

In [ ]:
correlations_Noise = powerPraatDF.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 1


Variability of Syllable Period   -0.083157
Name: Noise, dtype: float64

##Right vLPFC (0)

In [ ]:
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

##Left vLPFC (2 sig feats)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_vlPFC = powerPraatDF.corr()['Power: Left vlPFC'][8:].abs()
AcousticCorrelates99_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_vlPFC))
AcousticCorrelates99_Left_vlPFC.nlargest(len(AcousticCorrelates99_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 2


# of pauses                      0.137748
Variability of Pause Duration    0.115254
Name: Power: Left vlPFC, dtype: float64

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 2


# of pauses                      0.137748
Variability of Pause Duration    0.115254
Name: Power: Left vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(10).index.tolist()

['# of pauses', 'Variability of Pause Duration']

##Right OFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_OFC = powerPraatDF.corr()['Power: Right OFC'][8:].abs()
AcousticCorrelates99_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_OFC))
AcousticCorrelates99_Right_OFC.nlargest(len(AcousticCorrelates99_Right_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(10).index.tolist()

[]

##Left OFC (no sig feats)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_OFC = powerPraatDF.corr()['Power: Left OFC'][8:].abs()
AcousticCorrelates99_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_OFC))
AcousticCorrelates99_Left_OFC.nlargest(len(AcousticCorrelates99_Left_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Left OFC, dtype: float64)

#Sept 19 (

##read in files

In [ ]:
powerDF = pd.read_csv('2022-09-19-Power.csv')
del powerDF['Unnamed: 0']
powerDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,9.322238,4.995461,5.607341,3.754328
1,7.696407,7.638623,6.380779,7.017708
2,4.062073,4.581651,6.308548,9.426878
3,2.702953,3.492604,4.114555,7.404552
4,4.282622,5.795164,10.881743,3.557609


In [ ]:
praatAcousticFeatsDf = pd.read_csv('noModeratorSpeech_Sept19PerTwoSecondPraatFeatures.csv')
del praatAcousticFeatsDf['Unnamed: 0'], praatAcousticFeatsDf['Time (secs)']
praatAcousticFeatsDf.head()

,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1.5,0.397333,2.516779,1.475248,0.666667,0.242323,0.808,0.0,1
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
powerDF.shape, praatAcousticFeatsDf.shape

((1137, 4), (564, 9))

In [ ]:
# Reshape the array to have 50 columns (assuming the number of rows is a multiple of 50)
dfArray = powerDF[:-1].to_numpy()
reshaped_array = dfArray.reshape(-1, 2, dfArray.shape[1])

# Calculate the mean along axis 1 (average every 5 rows)
averaged_array = reshaped_array.mean(axis=1)

# Create a new DataFrame from the averaged array
columns = powerDF.columns  # Assuming you want to keep the same columns
resampled_powerDF = pd.DataFrame(averaged_array, columns=columns)
resampled_powerDF.shape

(568, 4)

In [ ]:
resampled_powerDF = resampled_powerDF.iloc[:praatAcousticFeatsDf.shape[0]]
resampled_powerDF.shape

(564, 4)

In [ ]:
powerPraatDF = pd.concat([resampled_powerDF, praatAcousticFeatsDf], axis=1)
powerPraatDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,8.509322,6.317042,5.994060,5.386018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,3.382513,4.037128,5.211552,8.415715,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,6.791152,7.366131,8.835148,5.475258,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,7.941317,-0.631314,11.155362,8.180236,1.5,0.397333,2.516779,1.475248,0.666667,0.242323,0.808,0.0,1
4,4.303380,6.855423,6.189374,10.970051,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.1084
rValFor95ConfidenceLevel = 0.0826

##random noise t-test (1)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random 1136-element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerPraatDF.shape[0])


In [ ]:
powerPraatDF['Noise'] = random_array

In [ ]:
powerPraatDF.corr()['Noise'][4:-1]#.abs()

speechrate(nsyll / dur)                               -0.040092
Average Syllable Duration (speakingtot/voicedcount)   -0.048510
articulation rate(nsyll / phonationtime)              -0.000968
Speech-to-pause ratio                                 -0.073595
Mean Syllable Period                                  -0.027754
Variability of Syllable Period                         0.001822
Mean Pause Duration                                    0.039269
Variability of Pause Duration                          0.065374
# of pauses                                            0.086078
Name: Noise, dtype: float64

In [ ]:
correlations_Noise = powerPraatDF.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 1


# of pauses    0.086078
Name: Noise, dtype: float64

##Right vLPFC (1)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_vlPFC = powerPraatDF.corr()['Power: Right vlPFC'][8:].abs()
AcousticCorrelates99_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_vlPFC))
AcousticCorrelates99_Right_vlPFC.nlargest(len(AcousticCorrelates99_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 1


Mean Syllable Period    0.092623
Name: Power: Right vlPFC, dtype: float64

##Left vLPFC (2)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_vlPFC = powerPraatDF.corr()['Power: Left vlPFC'][8:].abs()
AcousticCorrelates99_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_vlPFC))
AcousticCorrelates99_Left_vlPFC.nlargest(len(AcousticCorrelates99_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 2


Mean Pause Duration    0.100744
# of pauses            0.090510
Name: Power: Left vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(10).index.tolist()

['Mean Pause Duration', '# of pauses']

##Right OFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_OFC = powerPraatDF.corr()['Power: Right OFC'][8:].abs()
AcousticCorrelates99_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_OFC))
AcousticCorrelates99_Right_OFC.nlargest(len(AcousticCorrelates99_Right_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(10).index.tolist()

[]

##Left OFC (1)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_OFC = powerPraatDF.corr()['Power: Left OFC'][8:].abs()
AcousticCorrelates99_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_OFC))
AcousticCorrelates99_Left_OFC.nlargest(len(AcousticCorrelates99_Left_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 1


Mean Pause Duration    0.091506
Name: Power: Left OFC, dtype: float64

#Oct 4

##read in files

In [ ]:
powerDF = pd.read_csv('2022-10-04-Power-Amplitude.csv')
del powerDF['Unnamed: 0']
powerDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,6.407832,8.696524,10.436554,11.400055
1,6.523042,8.672893,8.723282,10.050933
2,6.641233,10.282269,8.287294,7.250344
3,3.958534,8.233730,10.518788,10.198656
4,8.917095,3.029191,9.664642,4.365085


In [ ]:
praatAcousticFeatsDf = pd.read_csv('noModeratorSpeech_Oct4PerTwoSecondPraatFeatures.csv')
del praatAcousticFeatsDf['Unnamed: 0'], praatAcousticFeatsDf['Time (secs)']
praatAcousticFeatsDf.head()

,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1.5,0.224,4.464286,0.506024,0.666667,0.123033,1.328,0.0,1


In [ ]:
powerDF.shape, praatAcousticFeatsDf.shape

((1154, 4), (553, 9))

In [ ]:
# Reshape the array to have 50 columns (assuming the number of rows is a multiple of 50)
dfArray = powerDF.to_numpy()
reshaped_array = dfArray.reshape(-1, 2, dfArray.shape[1])

# Calculate the mean along axis 1 (average every 5 rows)
averaged_array = reshaped_array.mean(axis=1)

# Create a new DataFrame from the averaged array
columns = powerDF.columns  # Assuming you want to keep the same columns
resampled_powerDF = pd.DataFrame(averaged_array, columns=columns)
resampled_powerDF.shape

(577, 4)

In [ ]:
resampled_powerDF = resampled_powerDF.iloc[:praatAcousticFeatsDf.shape[0]]
resampled_powerDF.shape

(553, 4)

In [ ]:
powerPraatDF = pd.concat([resampled_powerDF, praatAcousticFeatsDf], axis=1)
powerPraatDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,6.465437,8.684708,9.579918,10.725494,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,5.299883,9.258000,9.403041,8.724500,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,7.672788,6.339534,7.135017,6.149560,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,4.847016,10.642939,7.494360,10.591644,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,6.786832,8.923073,6.761784,8.954511,1.5,0.224,4.464286,0.506024,0.666667,0.123033,1.328,0.0,1


In [ ]:
powerPraatDF.shape

(553, 13)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.1095
rValFor95ConfidenceLevel = 0.0834

##random noise t-test (1)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random 1136-element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerPraatDF.shape[0])


In [ ]:
powerPraatDF['Noise'] = random_array

In [ ]:
powerPraatDF.corr()['Noise'][4:-1]#.abs()

speechrate(nsyll / dur)                               -0.024362
Average Syllable Duration (speakingtot/voicedcount)    0.011278
articulation rate(nsyll / phonationtime)              -0.033086
Speech-to-pause ratio                                  0.162307
Mean Syllable Period                                   0.026441
Variability of Syllable Period                         0.030397
Mean Pause Duration                                    0.002016
Variability of Pause Duration                          0.052220
# of pauses                                            0.033695
Name: Noise, dtype: float64

In [ ]:
correlations_Noise = powerPraatDF.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 1


Speech-to-pause ratio    0.162307
Name: Noise, dtype: float64

##Right vLPFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_vlPFC = powerPraatDF.corr()['Power: Right vlPFC'][8:].abs()
AcousticCorrelates99_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_vlPFC))
AcousticCorrelates99_Right_vlPFC.nlargest(len(AcousticCorrelates99_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(10).index.tolist()

[]

##Left vLPFC (1)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_vlPFC = powerPraatDF.corr()['Power: Left vlPFC'][8:].abs()
AcousticCorrelates99_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_vlPFC))
AcousticCorrelates99_Left_vlPFC.nlargest(len(AcousticCorrelates99_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 1


Variability of Syllable Period    0.090436
Name: Power: Left vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(10).index.tolist()

['Variability of Syllable Period']

##Right OFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_OFC = powerPraatDF.corr()['Power: Right OFC'][8:].abs()
AcousticCorrelates99_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_OFC))
AcousticCorrelates99_Right_OFC.nlargest(len(AcousticCorrelates99_Right_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(10).index.tolist()

[]

##Left OFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_OFC = powerPraatDF.corr()['Power: Left OFC'][8:].abs()
AcousticCorrelates99_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_OFC))
AcousticCorrelates99_Left_OFC.nlargest(len(AcousticCorrelates99_Left_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Left OFC, dtype: float64)

#Nov 15

##read in files

In [ ]:
powerDF = pd.read_csv('2022-11-15-Power.csv')
del powerDF['Unnamed: 0']
powerDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,8.462996,6.523769,4.619912,8.098723
1,5.854915,5.621985,7.196563,7.780724
2,9.335134,7.215692,7.966418,9.071778
3,4.709172,5.349554,7.623001,11.709882
4,5.242306,13.146638,7.908021,7.542927


In [ ]:
praatAcousticFeatsDf = pd.read_csv('noModeratorSpeech_Nov15PerTwoSecondPraatFeatures.csv')
del praatAcousticFeatsDf['Unnamed: 0'], praatAcousticFeatsDf['Time (secs)']
praatAcousticFeatsDf.head()

,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1.5,0.445333,2.245509,2.012048,0.666667,0.274689,0.664,0.0,1
4,3.0,0.230667,4.335260,2.246753,0.333333,0.254371,0.616,0.0,1


In [ ]:
powerDF.shape, praatAcousticFeatsDf.shape

((1143, 4), (550, 9))

In [ ]:
# Reshape the array to have 50 columns (assuming the number of rows is a multiple of 50)
dfArray = powerDF[:-1].to_numpy()
reshaped_array = dfArray.reshape(-1, 2, dfArray.shape[1])

# Calculate the mean along axis 1 (average every 5 rows)
averaged_array = reshaped_array.mean(axis=1)

# Create a new DataFrame from the averaged array
columns = powerDF.columns  # Assuming you want to keep the same columns
resampled_powerDF = pd.DataFrame(averaged_array, columns=columns)
resampled_powerDF.shape

(571, 4)

In [ ]:
resampled_powerDF = resampled_powerDF.iloc[:praatAcousticFeatsDf.shape[0]]
resampled_powerDF.shape

(550, 4)

In [ ]:
powerPraatDF = pd.concat([resampled_powerDF, praatAcousticFeatsDf], axis=1)
powerPraatDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,7.158956,6.072877,5.908238,7.939723,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,7.022153,6.282623,7.794710,10.390830,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,6.670946,11.387367,10.050743,9.421598,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,4.279018,13.527752,7.794479,10.313778,1.5,0.445333,2.245509,2.012048,0.666667,0.274689,0.664,0.0,1
4,6.869506,8.578950,7.288889,9.134564,3.0,0.230667,4.335260,2.246753,0.333333,0.254371,0.616,0.0,1


In [ ]:
powerPraatDF.shape

(550, 13)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.1098
rValFor95ConfidenceLevel = 0.0837

##random noise t-test (0)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random 1136-element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerPraatDF.shape[0])


In [ ]:
powerPraatDF['Noise'] = random_array

In [ ]:
powerPraatDF.corr()['Noise'][4:-1]#.abs()

speechrate(nsyll / dur)                               -0.006220
Average Syllable Duration (speakingtot/voicedcount)    0.037122
articulation rate(nsyll / phonationtime)               0.013672
Speech-to-pause ratio                                 -0.016617
Mean Syllable Period                                   0.042890
Variability of Syllable Period                        -0.024034
Mean Pause Duration                                    0.053085
Variability of Pause Duration                         -0.023963
# of pauses                                           -0.004222
Name: Noise, dtype: float64

In [ ]:
correlations_Noise = powerPraatDF.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Noise, dtype: float64)

##Right vLPFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_vlPFC = powerPraatDF.corr()['Power: Right vlPFC'][8:].abs()
AcousticCorrelates99_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_vlPFC))
AcousticCorrelates99_Right_vlPFC.nlargest(len(AcousticCorrelates99_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(10).index.tolist()

[]

##Left vLPFC (3)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_vlPFC = powerPraatDF.corr()['Power: Left vlPFC'][8:].abs()
AcousticCorrelates99_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_vlPFC))
AcousticCorrelates99_Left_vlPFC.nlargest(len(AcousticCorrelates99_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 1


Mean Pause Duration    0.16328
Name: Power: Left vlPFC, dtype: float64

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 3


Mean Pause Duration               0.163280
Mean Syllable Period              0.109451
Variability of Syllable Period    0.102861
Name: Power: Left vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(10).index.tolist()

['Mean Pause Duration',
 'Mean Syllable Period',
 'Variability of Syllable Period']

##Right OFC (1)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_OFC = powerPraatDF.corr()['Power: Right OFC'][8:].abs()
AcousticCorrelates99_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_OFC))
AcousticCorrelates99_Right_OFC.nlargest(len(AcousticCorrelates99_Right_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 1


# of pauses    0.086592
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(10).index.tolist()

['# of pauses']

##Left OFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_OFC = powerPraatDF.corr()['Power: Left OFC'][8:].abs()
AcousticCorrelates99_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_OFC))
AcousticCorrelates99_Left_OFC.nlargest(len(AcousticCorrelates99_Left_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Left OFC, dtype: float64)

#Feb 27 Experiment 1 (Waiting for Sameer)

##read in files

In [ ]:
powerDF = pd.read_csv('2023-02-27-Experiment1-Power-Amplitude.csv')
del powerDF['Unnamed: 0']
powerDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,5.686801,9.739613,8.195255,8.018249
1,9.301764,10.138419,5.699974,8.329905
2,3.541367,9.768278,4.449306,13.404476
3,8.466536,6.540455,10.163884,11.027338
4,11.272942,8.775278,9.911861,12.002101


In [ ]:
praatAcousticFeatsDf = pd.read_csv('Feb27ExperimentOnePerTwoSecondPraatFeatures.csv')
del praatAcousticFeatsDf['Unnamed: 0'], praatAcousticFeatsDf['Time (secs)']
praatAcousticFeatsDf.head()

,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
powerDF.shape, praatAcousticFeatsDf.shape

((414, 4), (190, 9))

In [ ]:
dfArray = powerDF.to_numpy()
reshaped_array = dfArray.reshape(-1, 2, dfArray.shape[1])

averaged_array = reshaped_array.mean(axis=1)

# Create a new DataFrame from the averaged array
columns = powerDF.columns  # Assuming you want to keep the same columns
resampled_powerDF = pd.DataFrame(averaged_array, columns=columns)
resampled_powerDF.shape

(207, 4)

In [ ]:
resampled_powerDF = resampled_powerDF.iloc[:praatAcousticFeatsDf.shape[0]]
resampled_powerDF.shape

(190, 4)

In [ ]:
powerPraatDF = pd.concat([resampled_powerDF, praatAcousticFeatsDf], axis=1)
powerPraatDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,7.494282,9.939016,6.947614,8.174077,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,6.003951,8.154367,7.306595,12.215907,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,9.370430,8.363272,10.352769,13.243804,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,5.896198,6.918870,7.700420,10.774696,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,4.039299,6.548344,9.438954,9.933403,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
powerPraatDF.shape

(190, 13)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

In [ ]:
rValFor99ConfidenceLevel = 0.1867
rValFor95ConfidenceLevel = 0.1425

##random noise t-test (0)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random 1136-element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerPraatDF.shape[0])


In [ ]:
powerPraatDF['Noise'] = random_array

In [ ]:
powerPraatDF.corr()['Noise'][4:-1]#.abs()

speechrate(nsyll / dur)                                0.060362
Average Syllable Duration (speakingtot/voicedcount)    0.085963
articulation rate(nsyll / phonationtime)               0.068130
Speech-to-pause ratio                                  0.215650
Mean Syllable Period                                   0.069007
Variability of Syllable Period                         0.022844
Mean Pause Duration                                   -0.077227
Variability of Pause Duration                         -0.035560
# of pauses                                           -0.037585
Name: Noise, dtype: float64

In [ ]:
correlations_Noise = powerPraatDF.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 1


Speech-to-pause ratio    0.21565
Name: Noise, dtype: float64

##Right vLPFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_vlPFC = powerPraatDF.corr()['Power: Right vlPFC'][8:].abs()
AcousticCorrelates99_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_vlPFC))
AcousticCorrelates99_Right_vlPFC.nlargest(len(AcousticCorrelates99_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(10).index.tolist()

[]

##Left vLPFC (2)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_vlPFC = powerPraatDF.corr()['Power: Left vlPFC'][8:].abs()
AcousticCorrelates99_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_vlPFC))
AcousticCorrelates99_Left_vlPFC.nlargest(len(AcousticCorrelates99_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 2


Variability of Syllable Period    0.153769
Mean Syllable Period              0.145622
Name: Power: Left vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(10).index.tolist()

[]

##Right OFC

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_OFC = powerPraatDF.corr()['Power: Right OFC'][8:].abs()
AcousticCorrelates99_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_OFC))
AcousticCorrelates99_Right_OFC.nlargest(len(AcousticCorrelates99_Right_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(10).index.tolist()

[]

##Left OFC

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_OFC = powerPraatDF.corr()['Power: Left OFC'][8:].abs()
AcousticCorrelates99_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_OFC))
AcousticCorrelates99_Left_OFC.nlargest(len(AcousticCorrelates99_Left_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Left OFC, dtype: float64)

#Feb 27 Experiment 5 (Waiting for Sameer)

##read in files

In [ ]:
powerDF = pd.read_csv('2023-02-27-Experiment5-Power.csv')
del powerDF['Unnamed: 0']
powerDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,6.040388,9.369975,11.474579,9.765721
1,4.740961,11.204577,9.600362,9.777031
2,4.904171,8.815254,4.404419,11.895500
3,5.530016,7.770287,8.693739,10.962857
4,6.540160,11.214869,5.332976,10.099922


In [ ]:
praatAcousticFeatsDf = pd.read_csv('Feb27ExperimentFivePerTwoSecondPraatFeatures.csv')
del praatAcousticFeatsDf['Unnamed: 0'], praatAcousticFeatsDf['Time (secs)']
praatAcousticFeatsDf.head()

,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2.0,0.50,2.000000,NaN,0.500000,0.393564,0.00,0.00,1
2,1.5,0.56,1.785714,5.250000,0.666667,0.331196,0.16,0.32,2
3,0.0,NaN,0.000000,0.612903,0.000000,0.000000,1.24,0.00,1
4,2.5,0.40,2.500000,NaN,0.400000,0.479806,0.00,0.00,1


In [ ]:
powerDF.shape, praatAcousticFeatsDf.shape

((624, 4), (182, 9))

In [ ]:
powerDF=powerDF.iloc[0:372]
powerDF.shape

(372, 4)

In [ ]:
dfArray = powerDF.to_numpy()
reshaped_array = dfArray.reshape(-1, 2, dfArray.shape[1])

averaged_array = reshaped_array.mean(axis=1)

# Create a new DataFrame from the averaged array
columns = powerDF.columns  # Assuming you want to keep the same columns
resampled_powerDF = pd.DataFrame(averaged_array, columns=columns)
resampled_powerDF.shape

(186, 4)

In [ ]:
resampled_powerDF = resampled_powerDF.iloc[:praatAcousticFeatsDf.shape[0]]
resampled_powerDF.shape

(182, 4)

In [ ]:
powerPraatDF = pd.concat([resampled_powerDF, praatAcousticFeatsDf], axis=1)
powerPraatDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC,speechrate(nsyll / dur),Average Syllable Duration (speakingtot/voicedcount),articulation rate(nsyll / phonationtime),Speech-to-pause ratio,Mean Syllable Period,Variability of Syllable Period,Mean Pause Duration,Variability of Pause Duration,# of pauses
0,5.390675,10.287276,10.537470,9.771376,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,5.217093,8.292771,6.549079,11.429179,2.0,0.50,2.000000,NaN,0.500000,0.393564,0.00,0.00,1
2,6.638462,8.456553,5.166038,9.499027,1.5,0.56,1.785714,5.250000,0.666667,0.331196,0.16,0.32,2
3,6.675131,10.542879,6.596650,14.070357,0.0,NaN,0.000000,0.612903,0.000000,0.000000,1.24,0.00,1
4,6.869453,10.128801,9.053532,12.241157,2.5,0.40,2.500000,NaN,0.400000,0.479806,0.00,0.00,1


In [ ]:
powerPraatDF.shape

(182, 13)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

In [ ]:
rValFor99ConfidenceLevel = 0.1885
rValFor95ConfidenceLevel = 0.144

##random noise t-test (2)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(35)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random 1136-element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerPraatDF.shape[0])


In [ ]:
powerPraatDF['Noise'] = random_array

In [ ]:
powerPraatDF.corr()['Noise'][4:-1]#.abs()

speechrate(nsyll / dur)                               -0.024597
Average Syllable Duration (speakingtot/voicedcount)   -0.223240
articulation rate(nsyll / phonationtime)               0.152776
Speech-to-pause ratio                                 -0.054681
Mean Syllable Period                                  -0.072172
Variability of Syllable Period                         0.124826
Mean Pause Duration                                   -0.022375
Variability of Pause Duration                          0.097475
# of pauses                                            0.086312
Name: Noise, dtype: float64

In [ ]:
correlations_Noise = powerPraatDF.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 2


articulation rate(nsyll / phonationtime)    0.155740
Speech-to-pause ratio                      -0.207571
Name: Noise, dtype: float64

##Right vLPFC (0)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_vlPFC = powerPraatDF.corr()['Power: Right vlPFC'][8:].abs()
AcousticCorrelates99_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_vlPFC))
AcousticCorrelates99_Right_vlPFC.nlargest(len(AcousticCorrelates99_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[correlations_Right_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Right vlPFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(10).index.tolist()

[]

##Left vLPFC

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_vlPFC = powerPraatDF.corr()['Power: Left vlPFC'][8:].abs()
AcousticCorrelates99_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_vlPFC))
AcousticCorrelates99_Left_vlPFC.nlargest(len(AcousticCorrelates99_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left vlPFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[correlations_Left_vlPFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 0


Series([], Name: Power: Left vlPFC, dtype: float64)

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(10).index.tolist()

[]

##Right OFC

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Right_OFC = powerPraatDF.corr()['Power: Right OFC'][8:].abs()
AcousticCorrelates99_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Right_OFC))
AcousticCorrelates99_Right_OFC.nlargest(len(AcousticCorrelates99_Right_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Right OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[correlations_Right_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 2


Variability of Pause Duration     0.151306
Variability of Syllable Period    0.145986
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(10).index.tolist()

['Variability of Pause Duration', 'Variability of Syllable Period']

##Left OFC

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
correlations_Left_OFC = powerPraatDF.corr()['Power: Left OFC'][8:].abs()
AcousticCorrelates99_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor99ConfidenceLevel]
print("# of acoustic features where p < 0.01:", len(AcousticCorrelates99_Left_OFC))
AcousticCorrelates99_Left_OFC.nlargest(len(AcousticCorrelates99_Left_OFC)) # for sorting

# of acoustic features where p < 0.01: 0


Series([], Name: Power: Left OFC, dtype: float64)

In [ ]:
# Assuming powerAmplitudeAllLLDdf is your DataFrame
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[correlations_Left_OFC > rValFor95ConfidenceLevel]
print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 1


Mean Syllable Period    0.182978
Name: Power: Left OFC, dtype: float64